In [4]:
import gzip
from collections import defaultdict
import random 
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [5]:
print('reading data...')
data = list(readGz('train.json.gz'))
print('done')

reading data...
done


In [6]:
known_ub_pair = [(d['userID'],d['businessID']) for d in data]  #get known user,business pair from original 200,000 training data
known_user = [d['userID'] for d in data]
known_business = [d['businessID'] for d in data]
iden_user = list(set(known_user))
iden_business = list(set(known_business))


In [83]:
userRatings = defaultdict(list)
for d in data:
    user,business = d['userID'],d['businessID']
    userRatings[user].append(d['rating'])
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u])/len(userRatings[u])

In [8]:
#create business-category pairs(duplicate category exist)
busiCat_set = defaultdict(set)
for b in iden_business:
    cat = [d['categories'] for d in data if d['businessID'] == b]
    combine_cat = [j for i in cat for j in i]
    for i in combine_cat: 
        busiCat_set[b].add(i)

In [9]:
#create business_user and user_business pairs
user_business = defaultdict(list)
business_user = defaultdict(list)
for (u,b) in known_ub_pair:
    user_business[u].append(b)
    business_user[b].append(u)
for u in user_business:
    user_business[u] = list(set(user_business[u]))
for b in business_user:
    business_user[b] = list(set(business_user[b]))

In [10]:
#get the list of popular business
businessCount = defaultdict(int)
totalPurchase = 0
for (u,b) in known_ub_pair:
    businessCount[b] += 1
    totalPurchase += 1
mostPopular = [(businessCount[x],x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()
popular_business=set()  #set of 58% popular business
count = 0
for ic,i in mostPopular:
    count +=ic
    popular_business.add(i)
    if count>totalPurchase*5.8/10:break
    

In [91]:
#get the business category of each user
userCat_set = defaultdict(set)
for r in data:
    if 'categories' in r:
        for c in r['categories']:
            if r['rating']>=userAverage[r['userID']]:
                userCat_set[r['userID']].add(c)

In [94]:
userCat = defaultdict(set)
for r in data:
    if 'categories' in r:
        for c in r['categories']:
            userCat[r['userID']].add(c)

In [98]:
############
predictions = open('task1.txt','w')
for l in open('pairs_Visit.txt'):
    if l.startswith('userID'):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if u in iden_user:
        if b in iden_business:
            similar_business = []
            for elseuser in business_user[b]:
                sim = len(userCat_set[u].intersection(userCat_set[elseuser]))/len(userCat_set[u].union(userCat_set[elseuser]))
                if sim>=0.25:  #similarity beween users
                    for bb in user_business[elseuser]:
                        similar_business.append(bb)
            if b in similar_business:
                pre_predict_1 = 1
            else:
                pre_predict_1 = 0
            #################################
            sim_value = []
            for bbb in user_business[u]:
                sim = len(busiCat_set[b].intersection(busiCat_set[bbb]))/len(busiCat_set[b].union(busiCat_set[b]))
                sim_value.append(sim)
            sim_value.sort()
            sim_value.reverse()
            if len(sim_value)>1 and sim_value[1]>0.8:  #threshold for business in test and that of all the business user visited
                pre_predict_2 = 1
            else:
                pre_predict_2 = 0
            if pre_predict_1==1 and pre_predict_2 ==1:
                predict = 1
            else:
                predict = 0
                
                
        else:
            if len(userCat[u])>40:                  #
                predict = 1
            else:
                predict = 0
    else:
        if b in iden_business and b in popular_business:
            predict = 1
        else:
            predict = 0#random.randint(0,1)
    predictions.write(u + '-' + b + ',' + str(predict) + '\n')
predictions.close()      
    

In [ ]:
predictions = open('task1.txt','w')
for l in open('pairs_Visit.txt'):
    if l.startswith('userID'):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if u in iden_user and b in iden_business:
        similarity = []
        for u_b in user_business[u]:
            similarity.append(len(busiCat_set[b].intersection(busiCat_set[u_b]))/len(busiCat_set[b].union(busiCat_set[u_b])))
            similarity.sort()
            similarity.reverse()
            if similarity[0]>0.2:                   #
                predict = 1
            else:
                predict = 0
    if u in iden_user and b not in iden_business:
        if len(userCat_set[u])>25:                  #
            predict = 1
        else:
            predict = 0
    if u not in iden_user and b in iden_business:
        if b in popular_business:
            predict = 1
        else:
            predict = 0
    if u not in iden_user and b not in iden_business:
        predict = 0#random.randint(0,1)
    predictions.write(u + '-' + b + ',' + str(predict) + '\n')
predictions.close()      
    

In [ ]:
######################create business-category pairs(duplicate category exist)
busiCat = defaultdict(list)
for b in iden_business:
    cat = [d['categories'] for d in data if d['businessID'] == b]
    combine_cat = [j for i in cat for j in i]
    for i in combine_cat: 
        busiCat[b].append(i)
#######################calculate the similarity between every business pairs
sim_u = defaultdict(float)  
for n1 in range(len(iden_business)):
    for n2 in range(len(iden_business)):
        user1 = iden_business[n1]
        user2 = iden_business[n2]
        sim_u[(user1,user2)] = len(set(busiCat[user1]).intersection(set(busiCat[user2])))/len(set(busiCat[user1]).union(set(busiCat[user2])))